In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# set parameters
project_id = "apply-ds-test-297613"
current_date = (datetime.now() - timedelta(days=60)).strftime("%Y-%m-%d")
covid_county = "Cook County"
covid_state = "IL"

## Covid data

In [3]:
# create query for covid data
covid_query = """
    SELECT state, county_name, DATE(date) AS date, confirmed_cases, deaths
    FROM bigquery-public-data.covid19_usafacts.summary
    WHERE date='{0}' and state='{1}' and county_name='{2}'
""".format(current_date, covid_state, covid_county)

print(covid_query)


    SELECT state, county_name, DATE(date) AS date, confirmed_cases, deaths
    FROM bigquery-public-data.covid19_usafacts.summary
    WHERE date='2020-10-18' and state='IL' and county_name='Cook County'



In [4]:
# query covid data
covid = pd.read_gbq(covid_query, project_id=project_id, dialect='standard')
covid

,state,county_name,date,confirmed_cases,deaths
0,IL,Cook County,2020-10-18,163912.000000000,5357.000000000


## Chicago crime data

In [5]:
# create query for crime data
crime_query = """
    SELECT DATE(date) as date, primary_type, domestic, count(*) AS num_cases
    FROM bigquery-public-data.chicago_crime.crime
    WHERE DATE(date)='{0}'
    GROUP BY DATE(date), primary_type, domestic
""".format(current_date)

print(crime_query)


    SELECT DATE(date) as date, primary_type, domestic, count(*) AS num_cases
    FROM bigquery-public-data.chicago_crime.crime
    WHERE DATE(date)='2020-10-18'
    GROUP BY DATE(date), primary_type, domestic



## Merge datasets

In [6]:
# query Chicago crime data
crime = pd.read_gbq(crime_query, project_id=project_id, dialect='standard')
crime.head()

,date,primary_type,domestic,num_cases
0,2020-10-18,DECEPTIVE PRACTICE,False,75
1,2020-10-18,WEAPONS VIOLATION,False,19
2,2020-10-18,OTHER OFFENSE,False,12
3,2020-10-18,OFFENSE INVOLVING CHILDREN,False,2
4,2020-10-18,BATTERY,False,49


In [69]:
# create counts by date and domestic (yes/no)
crime \
    .groupby(["domestic"])["num_cases"] \
    .sum() \
    .reset_index() \
    .set_index("domestic") \
    .transpose()

domestic,False,True
num_cases,427,125
